In [1]:
# Cell 1: Imports and Setup
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import warnings
from rasterio.errors import NotGeoreferencedWarning
from sklearn.preprocessing import StandardScaler
from rasterio.windows import Window
from scipy.ndimage import generic_filter

# Suppress NotGeoreferencedWarning
warnings.simplefilter("ignore", NotGeoreferencedWarning)

# # Paths to the TIFF images for different years
image_paths = {
    '2018': 'dataset/2018/m_3912112_sw_10_060_20180718.tif',
    '2022': 'dataset/2022/m_3912112_sw_10_060_20220714.tif'
}


In [2]:
# Cell 2: Helper Functions
def read_tiff(filename):
    """
    Reads the entire TIFF image.
    """
    with rasterio.open(filename) as src:
        image = src.read()
    return image

def calculate_ndvi(nir, red):
    """Calculates NDVI using NIR and red bands."""
    return (nir.astype(float) - red.astype(float)) / (nir + red + 1e-8)

def add_texture(image, window_size=3):
    """Adds texture feature to the image."""
    texture = generic_filter(image, np.std, size=window_size)
    return np.dstack((image, texture))

In [ ]:
# Cell 3: Data Preparation
# Read subsets from all years
subsets = {year: read_tiff(path) for year, path in image_paths.items()}

# Calculate NDVI for all years
ndvis = {}
for year, subset in subsets.items():
    red, nir = subset[0], subset[3]  # Assuming bands are in order: R, G, B, NIR
    ndvis[year] = calculate_ndvi(nir, red)

# Prepare features and ground truth for each year
predictions = {}
ground_truths = {}

for year in ['2018', '2022']:
    rgb = np.moveaxis(subsets[year][:3], 0, -1)
    features = add_texture(rgb).reshape(-1, 4)
    
    if year == '2018':
        ground_truths[year] = np.where(ndvis['2018'] > 0.3, 1, 0).ravel()
    else:
        current_year_ndvi = ndvis['2022'] > 0.3
        previous_year_ndvi = ndvis['2018'] <= 0.3
        print("HERE")
        ground_truths[year] = (current_year_ndvi & previous_year_ndvi).astype(int).ravel()
        print("HERE 2")

    min_length = min(ground_truths[year].size, features.shape[0])
    features = features[:min_length]
    ground_truths[year] = ground_truths[year][:min_length]


In [ ]:
# Cell 4 Apply model
scaler = StandardScaler()

# Initialize predictions dictionary to store results for both years
predictions = {}

for year in ['2018', '2022']:
    # Scale features for each year (using features from the last processed year)
    features_scaled = scaler.fit_transform(features)

    X_train, X_test, y_train, y_test = train_test_split(features_scaled, ground_truths[year], test_size=0.2, random_state=42)

    # Random Forest Classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, class_weight="balanced", n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Store predictions for both years in the predictions dictionary
    predictions[f'{year}_rf'] = rf.predict(features_scaled)

    # Gradient Boosting Classifier
    from sklearn.utils.class_weight import compute_sample_weight
    
    # Compute sample weights to balance classes
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
    
    gb = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=10)
    gb.fit(X_train, y_train, sample_weight=sample_weights)
    
    # Store predictions for both years in the predictions dictionary
    predictions[f'{year}_gb'] = gb.predict(features_scaled)

In [ ]:
# Cell 5: Change Detection and Visualization
# Compute change detection (e.g., from 2018 to 2022)
deforestation_rf = (predictions['2018_rf'] == 1) & (predictions['2022_rf'] == 0)
reforestation_rf = (predictions['2018_rf'] == 0) & (predictions['2022_rf'] == 1)
no_change_rf = (predictions['2018_rf'] == predictions['2022_rf'])

deforestation_gb = (predictions['2018_gb'] == 1) & (predictions['2022_gb'] == 0)
reforestation_gb = (predictions['2018_gb'] == 0) & (predictions['2022_gb'] == 1)
no_change_gb = (predictions['2018_gb'] == predictions['2022_gb'])

# Calculate total pixels and pixel categories
total_pixels = ndvis['2018'].size

print("Random Forest Results:")
print(f"Deforested Pixels: {np.sum(deforestation_rf)} ({(np.sum(deforestation_rf) / total_pixels) * 100:.2f}%)")
print(f"Reforested Pixels: {np.sum(reforestation_rf)} ({(np.sum(reforestation_rf) / total_pixels) * 100:.2f}%)")
print(f"Unchanged Pixels: {np.sum(no_change_rf)} ({(np.sum(no_change_rf) / total_pixels) * 100:.2f}%)")

print("\nGradient Boosting Results:")
print(f"Deforested Pixels: {np.sum(deforestation_gb)} ({(np.sum(deforestation_gb) / total_pixels) * 100:.2f}%)")
print(f"Reforested Pixels: {np.sum(reforestation_gb)} ({(np.sum(reforestation_gb) / total_pixels) * 100:.2f}%)")
print(f"Unchanged Pixels: {np.sum(no_change_gb)} ({(np.sum(no_change_gb) / total_pixels) * 100:.2f}%)")

fig, axs = plt.subplots(3, 2, figsize=(20, 30))

# Plot original images
axs[0, 0].imshow(np.moveaxis(subsets['2018'][:3], 0, -1))
axs[0, 0].set_title('Original Image 2018')
axs[0, 0].axis('off')

axs[0, 1].imshow(np.moveaxis(subsets['2022'][:3], 0, -1))
axs[0, 1].set_title('Original Image 2022')
axs[0, 1].axis('off')

# Plot NDVI images
axs[1, 0].imshow(ndvis['2018'], cmap='RdYlGn', vmin=-1, vmax=1)
axs[1, 0].set_title('NDVI 2018')
axs[1, 0].axis('off')

axs[1, 1].imshow(ndvis['2022'], cmap='RdYlGn', vmin=-1, vmax=1)
axs[1, 1].set_title('NDVI 2022')
axs[1, 1].axis('off')

# Plot Random Forest results
rf_result = np.zeros(ndvis['2018'].shape, dtype=int)
rf_result[deforestation_rf.reshape(ndvis['2018'].shape)] = 1  # Red for deforestation
rf_result[reforestation_rf.reshape(ndvis['2018'].shape)] = 2  # Green for reforestation
rf_result[no_change_rf.reshape(ndvis['2018'].shape)] = 3  # Blue for no change

axs[2, 0].imshow(rf_result, cmap=plt.cm.colors.ListedColormap(['white', 'red', 'green', 'blue']))
axs[2, 0].set_title('Random Forest Results')
axs[2, 0].axis('off')

# Plot Gradient Boosting results
gb_result = np.zeros(ndvis['2018'].shape, dtype=int)
gb_result[deforestation_gb.reshape(ndvis['2018'].shape)] = 1  # Red for deforestation
gb_result[reforestation_gb.reshape(ndvis['2018'].shape)] = 2  # Green for reforestation
gb_result[no_change_gb.reshape(ndvis['2018'].shape)] = 3  # Blue for no change

axs[2, 1].imshow(gb_result, cmap=plt.cm.colors.ListedColormap(['white', 'red', 'green', 'blue']))
axs[2, 1].set_title('Gradient Boosting Results')
axs[2, 1].axis('off')

# Add a common legend
legend_elements = [plt.Rectangle((0,0),1,1, facecolor='red', edgecolor='none', label='Deforestation'),
                   plt.Rectangle((0,0),1,1, facecolor='green', edgecolor='none', label='Reforestation'),
                   plt.Rectangle((0,0),1,1, facecolor='blue', edgecolor='none', label='No Change'),
                   plt.Rectangle((0,0),1,1, facecolor='white', edgecolor='none', label='Unclassified')]

fig.legend(handles=legend_elements, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()
plt.show()

#Areas that were not forest in either year (e.g., water bodies, urban areas, or bare soil)

In [ ]:
# Cell 5: Model Evaluation
def print_metrics(y_true, y_pred, model_name):
    print(f"Model: {model_name}")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print("\n")

for year in ['2018', '2022']:
    print(f"Year: {year}")
    print_metrics(ground_truths[year], predictions[f'{year}_rf'], "Random Forest")
    print_metrics(ground_truths[year], predictions[f'{year}_gb'], "Gradient Boosting")

In [14]:
a = np.array([1, 2, 3])

b = np.where(a > 1, 1, 0)